In [1]:
import os
import glob
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

from concurrent import futures
from dateutil.relativedelta import relativedelta
from pandas_datareader import data as pdr
from scipy.stats import gaussian_kde

""" set output directory """
data_dir = "./data"
os.makedirs(data_dir, exist_ok=True)

In [2]:
""" datetime util """
now = dt.datetime.now()
lastday = now + relativedelta(months=0, days=-1)
firstday_of_this_month = dt.datetime(now.year, now.month, 1)
lastday_of_this_month = dt.datetime(now.year, now.month, 1) + relativedelta(months=1, days=-1)
firstday_of_last_month = dt.datetime(now.year, now.month, 1) + relativedelta(months=-1, days=0)
lastday_of_last_month = dt.datetime(now.year, now.month, 1) + relativedelta(months=0, days=-1)

In [3]:
""" set the download window """
start_date = "2017-01-01"
end_date = lastday.strftime('%Y-%m-%d')

# Data Loading

In [4]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

In [5]:
sp500_df = tables[0]
second_table = tables[1]
print(sp500_df.shape)

# rename symbol to escape symbol error
sp500_df["Symbol"] = sp500_df["Symbol"].map(lambda x: x.replace(".", "-"))
sp500_df.to_csv(f"{data_dir}/SP500_{end_date}.csv", index=False)
sp500_df = pd.read_csv(f"{data_dir}/SP500_{end_date}.csv")
print(sp500_df.shape)
sp500_tickers = list(sp500_df["Symbol"])
sp500_df.head()

(503, 8)
(503, 8)


,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [6]:
data_dir

'./data'

In [7]:
bad_names = [] # to keep track of failed queries

def download_stock(stock, end_date):
    try: 
        # download dataframe
        data = pdr.get_data_yahoo(stock, start=start_date, end=end_date)
        data['Name'] = stock
        output_name = f"{data_dir}//{end_date}_{stock}.csv"
        data.to_csv(output_name)
    
    except BaseException as e:
        bad_names.append(stock)
        print(e)
        print('bad: %s' % (stock))    

In [8]:
path_failed_queries = f'{data_dir}/failed_queries.txt'
if os.path.exists(path_failed_queries):
    with open(path_failed_queries) as f:
        failed_queries = f.read().split("\n")[:-1]
        sp500_tickers_ = failed_queries
else:
    sp500_tickers_ = sp500_tickers
print("number of stockes to download:", len(sp500_tickers_))

number of stockes to download: 503


In [9]:
print(type(sp500_tickers_), len(sp500_tickers_))
# sp500_tickers_

<class 'list'> 503


# Scraping

In [10]:
for ticker in sp500_tickers_:
    download_stock(ticker, end_date)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [11]:
# # set the maximum thread number
# max_workers = 20
# now = dt.datetime.now()

# # in case a smaller number of stocks than threads was passed in
# workers = min(max_workers, len(sp500_tickers_))

# with futures.ThreadPoolExecutor(workers) as executor:
#     res = executor.map(download_stock, sp500_tickers_)
    
# """ Save failed queries to a text file to retry """
# if len(bad_names) > 0:
#     with open(path_failed_queries, 'w') as outfile:
#         for name in bad_names:
#             outfile.write(name + "\n")
                          
# finish_time = dt.datetime.now()
# duration = finish_time - now
# minutes, seconds = divmod(duration.seconds, 60)
# print(f'The threaded script took {minutes} minutes and {seconds} seconds to run.')
# print(f'{len(bad_names)} stocks failed: ', bad_names)                          

# Preprocessing

In [12]:
historial_stock_data_files = glob.glob(f"{data_dir}/*.csv")
highest_day_list = []
for files in historial_stock_data_files:
    price = pd.read_csv(files, index_col="Date", parse_dates = True)
    ticker = os.path.splitext(os.path.basename(files))[0]
    price_close = price[["Close"]]
    highest_day = price_close.idxmax()[0]
    highest_price = price_close.max()[0]
    # print(f"{price_close}:{highest_day}:{highest_price}")
    highest_day_list.append(
        pd.DataFrame({"highest_day": [highest_day], "ticker": [ticker], "highest_price": highest_price }))

df = pd.concat(highest_day_list).reset_index(drop=True)
print(df.shape)
df.head()

ValueError: 'Date' is not in list

In [ ]:
df["highest_month"] = df["highest_day"].dt.to_period("M")
df = pd.merge(df, sp500_df[["Symbol", "GICS Sector", "GICS Sub-Industry"]],
              left_on='ticker', right_on='Symbol')

In [ ]:
df.count()

# Stock Analysis

## 최근 최고치를 경신한 주식

In [ ]:
df.sort_values("highest_day", ascending=False).head(20)

## 지난달 업종별 최고가 섹터

In [ ]:
industry_value_counts = df[df["highest_day"] >= "2022-12-01"]["GICS Sub-Industry"].value_counts()
fig, ax = plt.subplots(figsize=(20, 8))
ax.bar(industry_value_counts.index, industry_value_counts.values)
ax.set_xticklabels(industry_value_counts.index, rotation=90)
ax.set_xlabel("industry")
ax.set_ylabel("number of stocks")
plt.show()

In [ ]:
industry_value_counts[industry_value_counts.index.str.contains("Packaged Foods & Meats")]

In [ ]:
highest_day_count = df.groupby("highest_month").count()
highest_day_count["ticker"].plot()
plt.title("Number of stocks that reached new highs")
plt.ylabel("number of stocks")
plt.show()

In [ ]:
highest_day_count["ticker"].plot(marker=".")
plt.grid(axis='y')
plt.title("Number of stocks that reached new highs")
plt.xlim("2019-01-01", "2022-12-30")
plt.ylabel("number of stocks")
plt.show()

In [ ]:
ticker_list = ["GOOG", "AAPL", "FB", "AMZN", "MSFT", "TSLA", "NVDA"]
df[df["ticker"].isin(ticker_list)]

In [ ]:
industry_value_counts = df[df["highest_day"] <= "2021-12